In [1]:
from pathlib import Path

In [2]:
BUCKET: str = "bigdatateaching"
OLD_DIR: str = "reddit"
NEW_DIR: str = "reddit-project"

In [3]:
!aws s3 ls s3://{BUCKET}/{OLD_DIR}/ --recursive > {OLD_DIR}

In [4]:
!head {OLD_DIR}

2024-08-11 23:51:50  102408315 reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_1.parquet
2024-08-11 23:51:58  102955903 reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_10.parquet
2024-08-11 23:51:42  104833981 reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_100.parquet
2024-08-11 23:52:01  107551504 reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_101.parquet
2024-08-11 23:51:37  106506156 reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_102.parquet
2024-08-11 23:51:22  104921613 reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_103.parquet
2024-08-11 23:51:16  102952905 reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_104.parquet
2024-08-11 23:51:53   99885643 reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_105.parquet
2024-08-11 23:51:45  102357527 reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_106.parquet
2024-08-11 23:51:56  105036597 reddit/pa

In [5]:
!aws s3 ls s3://{BUCKET}/{NEW_DIR}/ --recursive > {NEW_DIR}

In [6]:
!head {NEW_DIR}

2024-10-06 04:10:53  102408315 reddit-project/reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_1.parquet
2024-10-06 04:11:02  102955903 reddit-project/reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_10.parquet
2024-10-06 04:11:11  104833981 reddit-project/reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_100.parquet
2024-10-06 04:11:20  107551504 reddit-project/reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_101.parquet
2024-10-06 04:11:29  106506156 reddit-project/reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_102.parquet
2024-10-06 04:11:38  104921613 reddit-project/reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_103.parquet
2024-10-06 04:11:47  102952905 reddit-project/reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_104.parquet
2024-10-06 15:02:07   99885643 reddit-project/reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_105.parquet
2024-10-06 04:11:55  102357

In [7]:
x = "reddit-project/reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_1.parquet"
"/".join(x.split("/")[2:])

'parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_1.parquet'

In [8]:
def handle_file(fname, replace_tld):
    print(f"handle_file, {fname}")
    # read contents
    lines = Path(fname).read_text().split("\n")
    
    # we have lines now, we only want lines containing the word reddit
    """
    2024-08-11 23:51:50  102408315 reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_1.parquet
    2024-08-11 23:51:58  102955903 reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_10.parquet
    """
    lines = [l for l in lines if 'reddit' in l]

    # we only want the filename portion of the line
    lines = [l.split()[3] for l in lines]

    # we only want the part to start from the parquet/ portion
    # because that is the file name that we can compare (the root dir name is reddit and reddit-parquet so we know those are different anyway)
    if replace_tld:
        lines = [l.replace(fname + "/", "") for l in lines]
    print(f"{fname}, lines={lines[:5]}")
    print(f"{fname} has {len(lines)} lines")
    return lines

In [9]:
old = handle_file(OLD_DIR, False)
new = handle_file(NEW_DIR, True)

handle_file, reddit
reddit, lines=['reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_1.parquet', 'reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_10.parquet', 'reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_100.parquet', 'reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_101.parquet', 'reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_102.parquet']
reddit has 4276 lines
handle_file, reddit-project
reddit-project, lines=['reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_1.parquet', 'reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_10.parquet', 'reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_100.parquet', 'reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_101.parquet', 'reddit/parquet/comments/yyyy=2023/mm=06/comments_RC_2023-06.zst_102.parquet']
reddit-project has 4255 lines


In [10]:
missing_lines = [l for l in old if l not in new]
print(f"there are {len(missing_lines)} lines that are in old but not in new")

there are 21 lines that are in old but not in new


In [11]:
print(missing_lines[:5])

['reddit/parquet/comments/yyyy=2023/mm=07/comments_RC_2023-07.zst_74.parquet', 'reddit/parquet/comments/yyyy=2023/mm=07/comments_RC_2023-07.zst_75.parquet', 'reddit/parquet/comments/yyyy=2023/mm=11/comments_RC_2023-11.zst_163.parquet', 'reddit/parquet/comments/yyyy=2023/mm=11/comments_RC_2023-11.zst_222.parquet', 'reddit/parquet/comments/yyyy=2023/mm=11/comments_RC_2023-11.zst_223.parquet']


In [12]:
Path("missing_files.txt").write_text("\n".join(missing_lines))

1580